In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

import glob
import pandas as pd
import tifffile
import os

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [3]:
!pip install timm segmentation_models_pytorch rasterio torchmetrics transformers
!apt install -y zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pyt

In [4]:
params = {
    'learning_rate': 0.001,
    'epochs': 30,
    'batch_size': 2,
    'num_workers': 8,
    'device': torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
    'div_factor':10,
    'final_div_factor':100,
    'threshhold':0.5
}

## Functions

In [5]:
def sort_paths_by_number(paths):
    return sorted(paths, key=lambda x: int(x.split('_')[-1].split('.tif')[0]))

In [6]:
def save_model(epoch,model,ckpt_path='./',name='tf_efficientnetv2_b2',val_iou=0):
    path = os.path.join(ckpt_path, '{}_wo_ca.pth'.format(name))
    torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)

def load_model(model,ckpt_path):
    state = torch.load(ckpt_path)
    model.load_state_dict(state)
    return model

In [7]:
def intersection_and_union(pred, true):
    """
    Calculates intersection and union for a batch of images.

    Args:
        pred (torch.Tensor): a tensor of predictions
        true (torc.Tensor): a tensor of labels

    Returns:
        intersection (int): total intersection of pixels
        union (int): total union of pixels
    """
    #pred = pred.squeeze(1)
    valid_pixel_mask = true.ne(255)  # valid pixel mask
    true = true.masked_select(valid_pixel_mask)
    pred = pred.masked_select(valid_pixel_mask)

    # Convert to CPU and NumPy for logical operations
    true = true.numpy()
    pred = pred.numpy()

    # Intersection and union totals
    intersection = np.logical_and(true, pred)
    union = np.logical_or(true, pred)
    return intersection.sum(), union.sum()

In [8]:
def calculate_f1_score(pred, true, threshold=0.5):
    """
    Compute the F1 score, also known as balanced F-score or F-measure.

    Parameters:
    - true: ground truth (correct) labels, a tensor of shape [N, H, W].
    - pred: predicted labels, a tensor of shape [N, 1, H, W].
    - threshold: the prediction threshold, defaults to 0.5.

    Returns:
    - F1 score
    """

    # Flatten the tensors
    true = true.numpy().flatten()
    pred = pred.numpy().flatten()

    # Calculate F1 score
    f1 = f1_score(true, pred)

    return f1

### Data

In [9]:
PATH_SRC = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/src/models'
PATH_OUTPUT = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/submit'
PATHS_VAL = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/evaluation_true_color/evaluation_true_color_*.tif'))
PATHS_VAL_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/sample/evaluation_mask_*.tif'))
PATHS_TRAIN = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_*.tif'))
PATHS_TRAIN_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_*.tif'))

In [10]:
class DataTransform():
    def __init__(self):
        self.data_transform = {
            "train": A.Compose(
                [
                  A.ShiftScaleRotate(shift_limit=0.0625,rotate_limit=15,p=0.5),
                  A.GridDistortion(p=0.35),
                  A.HorizontalFlip(p=0.5),
                  A.VerticalFlip(p=0.5),
                  A.GaussianBlur(p=0.25),
                  # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            ),
            "val": A.Compose(
                [
                  # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            )
        }

    def __call__(self, phase, img, mask):
      return self.data_transform[phase](image=img, mask=mask)

In [11]:
class CloudDataset(Dataset):
  def __init__(self, df, phase, transform):
    self.data = df
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def normalize_data(self, image_tiff):
    # Normalize the image for better visualization
    for i in range(3):  # Assuming the image has 3 channels
        image_tiff[:,:,i] = (image_tiff[:,:,i] - image_tiff[:,:,i].min()) / (image_tiff[:,:,i].max() - image_tiff[:,:,i].min())

    return image_tiff

  def __getitem__(self, index):
    row = self.data.iloc[index]
    true_color_PATH = row['true_color']
    mask_PATH = row['mask']
    img = tifffile.imread(true_color_PATH)
    img = self.normalize_data(img.astype(np.float32))
    mask = tifffile.imread(mask_PATH)
    mask = mask.astype(np.float32)

    img_and_mask = self.transform(self.phase, img, mask)
    img = img_and_mask["image"]
    mask = img_and_mask["mask"]

    return img, mask

In [12]:
df = pd.DataFrame({
    'true_color': PATHS_TRAIN,
    'mask': PATHS_TRAIN_MASK
})

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.head())

df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CloudDataset(df_train, phase="train", transform=DataTransform())
train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)


val_dataset = CloudDataset(df_val, phase="val", transform=DataTransform())
val_dataloader = DataLoader(
    val_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)

dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

                                                                                        true_color                                                                                 mask
0  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_0.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_0.tif
1  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_1.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_1.tif
2  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_2.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_2.tif
3  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_3.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_3.tif
4  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/trai

<ipython-input-12-44df3b6a699d>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# Models

UNet_b1

In [13]:
from timm.models.efficientnet import *
import segmentation_models_pytorch as smp
import torch.nn as nn
import yaml

class UNet_B1(nn.Module):
  def __init__(self):
    super(UNet_B1, self).__init__()

    aux_params=dict(
        pooling = 'avg',             # one of 'avg', 'max'
        dropout = 0.3,               # dropout ratio, default is None
        activation = None,           # activation function, default is None
        classes = 1,
    )

    self.unet = smp.Unet(
        encoder_name = 'timm-efficientnet-b1',        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights = 'imagenet',      # use `imagenet` pre-trained weights for encoder initialization
        in_channels = 3,                          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        decoder_attention_type = None,            # model output channels (number of classes in your dataset)
        classes = 1,
        aux_params = aux_params
    )

  def forward(self, image):
    batch_size = len(image)
    mask,logit = self.unet(image)
    return mask

model_unet_b1 = UNet_B1()

Downloading: "https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b1-5c1377c4.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b1-5c1377c4.pth
100%|██████████| 30.1M/30.1M [00:01<00:00, 15.9MB/s]


UNet_b2

In [14]:
from segmentation_models_pytorch.decoders.unet.model import UnetDecoder
from segmentation_models_pytorch.base import SegmentationHead
import torch
import torch.nn as nn
import timm

class UNet_B2(nn.Module):
    def __init__(self):
        super(UNet_B2, self).__init__()
        norm_cfg = dict(type='BN', requires_grad=True)
        self.backbone = timm.create_model('tf_efficientnetv2_b2', features_only=True,
                                          out_indices=[0,1,2,3],pretrained=True)
        self.decode_head = UnetDecoder(
                            encoder_channels=[16, 32, 56, 120],
                            decoder_channels=[16, 32, 56, 120],
                            n_blocks=4,
                            use_batchnorm=True,
                            center=False,
                            attention_type=None
                        )

        self.segment_classifier = SegmentationHead(56,1,upsampling=4)


    def forward(self,image):
        x = self.backbone(image)
        x=self.decode_head(*x)
        x=self.segment_classifier(x)
        x = F.interpolate(x, image.shape[-2:], mode="bilinear", align_corners=True)
        return x

model_unet_b2 = UNet_B2()

Segformer_b1

In [15]:
from transformers import SegformerForSemanticSegmentation
import torch.nn as nn

class Segformer(nn.Module):
    def __init__(self):
        super(Segformer, self).__init__()

        self.segformer = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b1-finetuned-ade-512-512')
        self.segformer.decode_head.classifier = nn.Conv2d(256,1,kernel_size=1)
    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        image = image
        mask = self.segformer(image).logits
        mask = F.interpolate(mask, image.shape[-2:], mode="bilinear", align_corners=True)
        mask = mask.squeeze(1)

        return mask


model_segformer = Segformer()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Testing and Submition

In [16]:
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

PATH_UNET_B1 = f'{PATH_SRC}/unet_effnet_b1_wo_ca.pth'
PATH_UNET_B2 = f'{PATH_SRC}/tf_efficientnetv2_b2_wo_ca.pth'
PATH_SEGFORMER = f'{PATH_SRC}/segformer_wo_ca.pth'

unet_b1 = load_model(model_unet_b1, PATH_UNET_B1)
unet_b1 = unet_b1.to(params['device'])

unet_b2 = load_model(model_unet_b2, PATH_UNET_B2)
unet_b2 = unet_b2.to(params['device'])

segformer = load_model(model_segformer, PATH_SEGFORMER)
segformer = segformer.to(params['device'])

model_list = [unet_b1, unet_b2, segformer]

In [17]:
total_intersection=0
total_union=0
total_f1=0
total = 0

pbar = tqdm(enumerate(dataloaders_dict['val']),
                      total=len(dataloaders_dict['val']),
                      leave=True,
                      dynamic_ncols=True)

for i, (img, mask) in pbar:
    preds = torch.zeros((img.shape[0],img.shape[2],img.shape[3]))

    img = img.to(params['device'])

    for model in model_list:
      with torch.no_grad():
        output = model(img)
        output = output.squeeze(dim=1)
        pred = output.sigmoid()
        preds += pred.cpu().numpy()


    preds /= len(model_list)
    preds = preds > params['threshhold']
    mask = mask.detach().cpu()

    plt.imshow(img[0][0], cmap='gray')
    plt.colorbar()
    plt.title("pred")
    plt.show()

    plt.imshow(preds[0][0], cmap='gray')
    plt.colorbar()
    plt.title("pred")
    plt.show()

    plt.imshow(mask[0], cmap='gray')
    plt.colorbar()
    plt.title("mask")
    plt.show()

    intersection, union = intersection_and_union(preds, mask)
    f1 = calculate_f1_score(preds, mask)

    total_intersection += intersection
    total_union += union
    iou = total_intersection/total_union
    total_f1 += f1
    total += 1

    pbar.set_postfix({'iou': iou, 'f1': total_f1/total})

    # free GPU
    del img
    del mask


print(f'iou={iou} f1={total_f1/total}')

100%|██████████| 100/100 [01:28<00:00,  1.13it/s, iou=0.696, f1=0.71]

iou=0.6958404755515673 f1=0.7097455593832152


### Results
UNet B1  +  UNet B2  +  Segformer:

iou=0.7789826076281865 f1=0.8706964983621159

batch size: 24

iou=0.8278460618318396 f1=0.8971580739040275



---


UNet B1 50Epoch  +  UNet B2  +  Segformer:

iou=0.789498195593447 f1=0.8689647430529251


---

UNet B1 1000x1000  +  UNetB2 Crop  +  Segformer:

iou=0.6859368933180818 f1=0.6841173318602332



## Submit

In [67]:
df_sub = pd.DataFrame({
    'true_color': PATHS_VAL,
    'mask': PATHS_VAL_MASK
})

sub_dataset = CloudDataset(df_sub, phase="val", transform=DataTransform())
sub_dataloader = DataLoader(
    sub_dataset,
    batch_size=params['batch_size'],
    shuffle=False,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)


for i, (img, mask) in tqdm(enumerate(sub_dataloader), total=len(sub_dataloader)):
    preds = np.zeros((img.shape[0],img.shape[2],img.shape[3]))

    for model in model_list:
      with torch.no_grad():
        img = img.to(params['device'])
        output = model(img)
        output = output.squeeze(dim=1)
        pred = output.sigmoid()
        preds += pred.cpu().numpy()

    preds /= len(model_list)


    for batch_i, pred in enumerate(preds):
        # plt.figure(figsize=(6, 6))

        PATH = df_sub['true_color'].iloc[i*params['batch_size'] + batch_i]
        fname = os.path.basename(PATH)
        # fname_wo_ext = fname.split('.')[0]

        # plt.title(f'Prediction: {fname}')
        # plt.imshow(pred, cmap='gray', vmin=0, vmax=1)
        # plt.colorbar()
        # plt.savefig(f"{PATH_OUTPUT}/predict_{fname_wo_ext}.png")

        # if i == 0:
        #     plt.show();

        # plt.clf()
        # plt.close()

        # post process
        pred_sub = cv2.resize(pred, (1000, 1000), interpolation=0)
        pred_sub = (pred_sub > 0.5).astype(np.uint8)

        idx = fname.split("_")[-1]

        tifffile.imwrite(f"{PATH_OUTPUT}/evaluation_mask_{idx}", pred_sub)

100%|██████████| 42/42 [09:38<00:00, 13.77s/it]


In [68]:
import os
PATH_ZIP = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/finished_submits/submit_trained_batchsize24.zip'
PATH_SUBMITS = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/submit/evaluation_mask_*.tif'

os.system(f'zip -j {PATH_ZIP} {PATH_SUBMITS}')

0